In [1]:
import pandas as pd

In [6]:
import pandas as pd

# Load the already merged minute-level data
df = pd.read_csv("data/processed/XAUUSD/combined_data.csv", parse_dates=['timestamp'], index_col='timestamp')
if 'ATR' in df.columns:
    df.drop(columns='ATR', inplace=True)

# Resample to daily OHLC
daily_ohlc = df.resample('1D').agg({
    'high': 'max',
    'low': 'min',
    'close': 'last'
})

# Calculate previous day's close
daily_ohlc['prev_close'] = daily_ohlc['close'].shift(1)

# Calculate True Range (TR)
daily_ohlc['true_range'] = daily_ohlc.apply(
    lambda row: max(
        row['high'] - row['low'],
        abs(row['high'] - row['prev_close']),
        abs(row['low'] - row['prev_close'])
    ) if pd.notnull(row['prev_close']) else None,
    axis=1
)

#Calculate 14-day ATR (excluding current day via shift)
daily_ohlc['atr_14'] = daily_ohlc['true_range'].rolling(window=14).mean().shift(1)

# Forward-fill daily values into minute-level data
df['daily_high'] = daily_ohlc['high'].reindex(df.index, method='ffill')
df['daily_low'] = daily_ohlc['low'].reindex(df.index, method='ffill')
df['daily_close'] = daily_ohlc['close'].reindex(df.index, method='ffill')
df['true_range'] = daily_ohlc['true_range'].reindex(df.index, method='ffill')
df['atr_14'] = daily_ohlc['atr_14'].reindex(df.index, method='ffill')

# Overwrite the original file
df.to_csv("data/processed/XAUUSD/combined_data.csv")

print("True Range and 14-day ATR (based on previous days) added — combined_data.csv has been overwritten.")


True Range and 14-day ATR (based on previous days) added — combined_data.csv has been overwritten.
